# Analisi dell'impatto ambientale della produzione di cibi e mangimi
### In base ai dati raccolti dall' Organizzazione delle Nazioni Unite capiremo quali sono gli alimenti che richiedono più risorse naturali in modo tale da poter spostare il focus produttivo su alimenti più sostenibili per l'ambiente.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re as regex


FAO_df = pd.read_csv("data/FAO.csv", encoding='latin-1')
Food_Production_df = pd.read_csv("data/Food_Production.csv")
FAO_df["Type"] = ""

## Visualizzazione dei tipi di dato a disposizione

In [2]:
FAO_df.columns

Index(['Area Abbreviation', 'Area Code', 'Area', 'Item Code', 'Item',
       'Element Code', 'Element', 'Unit', 'latitude', 'longitude', 'Y1961',
       'Y1962', 'Y1963', 'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969',
       'Y1970', 'Y1971', 'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977',
       'Y1978', 'Y1979', 'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985',
       'Y1986', 'Y1987', 'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993',
       'Y1994', 'Y1995', 'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001',
       'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009',
       'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Type'],
      dtype='object')

In [3]:
Food_Production_df.columns

Index(['Food product', 'Land use change', 'Animal Feed', 'Farm', 'Processing',
       'Transport', 'Packging', 'Retail', 'Total_emissions',
       'Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal)',
       'Eutrophying emissions per kilogram (gPO₄eq per kilogram)',
       'Eutrophying emissions per 100g protein (gPO₄eq per 100 grams protein)',
       'Freshwater withdrawals per 1000kcal (liters per 1000kcal)',
       'Freshwater withdrawals per 100g protein (liters per 100g protein)',
       'Freshwater withdrawals per kilogram (liters per kilogram)',
       'Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)',
       'Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein)',
       'Land use per 1000kcal (m² per 1000kcal)',
       'Land use per kilogram (m² per kilogram)',
       'Land use per 100g protein (m² per 100g protein)',
       'Scarcity-weighted water use per kilogram (liters per kilogram)',
       'Scarcity-weighted water use per 100g protein 

## Campioni di valori presenti nei datasets

In [4]:
FAO_df

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Type
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360,
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89,
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,181,Zimbabwe,2948,Milk - Excluding Butter,5142,Food,1000 tonnes,-19.02,29.15,...,357.0,359.0,356.0,341.0,385.0,418.0,457.0,426,451,
21473,ZWE,181,Zimbabwe,2960,"Fish, Seafood",5521,Feed,1000 tonnes,-19.02,29.15,...,4.0,9.0,6.0,9.0,5.0,15.0,15.0,15,15,
21474,ZWE,181,Zimbabwe,2960,"Fish, Seafood",5142,Food,1000 tonnes,-19.02,29.15,...,14.0,17.0,14.0,15.0,18.0,29.0,40.0,40,40,
21475,ZWE,181,Zimbabwe,2961,"Aquatic Products, Other",5142,Food,1000 tonnes,-19.02,29.15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,


In [5]:
Food_Production_df

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),...,Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,...,371.076923,482.4,0.945482,1.907692,2.897446,7.60,5.846154,18786.2,14450.923080,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.80,3.943662,49576.3,69825.774650,13449.891480
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.3,4.754098,...,347.647059,59.1,0.628415,2.705882,1.202186,0.88,5.176471,2754.2,16201.176470,3762.568306
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0,0.9,0.708419,...,NaN,0.0,1.355236,14.666667,1.858316,1.81,20.111111,0.0,NaN,NaN
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0,2.6,4.820513,...,NaN,620.1,0.911681,NaN,0.581197,2.04,NaN,16438.6,NaN,4683.361823
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0,1.4,1.541311,...,NaN,217.7,0.515670,NaN,0.521368,1.83,NaN,9493.3,NaN,2704.643875
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0,1.6,5.008798,...,203.503036,435.7,0.524927,0.836058,4.565982,15.57,7.272303,22477.4,10498.552080,NaN


# Fenomeni e colonne

### Uno dei fenomeni da tenere sicuramente in considerazione è l'eutrofizzazione (Eutrophying) che porta alla morte di flora e fauna marina. Quindi sarà un dato che prenderemo in considerazione.
### Le colonne dove non è segnata l'unità di misura a livello di consumo ci si riferisce al consumo in KG di CO2

# Colonne del primo dataset
### Del primo dataset andremo a considerare le seguenti colonne:
#### - Area Abbreviation -> Abbreviazione dell'area
#### - Area -> Area scritta per esteso
#### - Item -> Prodotto
#### - Element -> Mangime o Cibo
#### - Unit -> Unità di misura per la produzione
#### - Y* (Tutte le colonne che iniziano con Y) -> Quanto è stato prodotto ogni anno

In [6]:
# Selezione delle colonne del primo dataset
colonne_interessate = ["Area Abbreviation", "Area", "Item", "Element", "Unit", "Type"]
FAO_df_colonne_interessate = [ colonna for colonna in FAO_df.columns if colonna.startswith("Y") or (colonna in colonne_interessate)]
FAO_df[FAO_df_colonne_interessate]

,Area Abbreviation,Area,Item,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Type
0,AFG,Afghanistan,Wheat and products,Food,1000 tonnes,1928.0,1904.0,1666.0,1950.0,2001.0,...,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895,
1,AFG,Afghanistan,Rice (Milled Equivalent),Food,1000 tonnes,183.0,183.0,182.0,220.0,220.0,...,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422,
2,AFG,Afghanistan,Barley and products,Feed,1000 tonnes,76.0,76.0,76.0,76.0,76.0,...,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360,
3,AFG,Afghanistan,Barley and products,Food,1000 tonnes,237.0,237.0,237.0,238.0,238.0,...,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89,
4,AFG,Afghanistan,Maize and products,Feed,1000 tonnes,210.0,210.0,214.0,216.0,216.0,...,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21472,ZWE,Zimbabwe,Milk - Excluding Butter,Food,1000 tonnes,230.0,232.0,234.0,238.0,234.0,...,357.0,359.0,356.0,341.0,385.0,418.0,457.0,426,451,
21473,ZWE,Zimbabwe,"Fish, Seafood",Feed,1000 tonnes,27.0,25.0,27.0,23.0,27.0,...,4.0,9.0,6.0,9.0,5.0,15.0,15.0,15,15,
21474,ZWE,Zimbabwe,"Fish, Seafood",Food,1000 tonnes,6.0,6.0,6.0,8.0,9.0,...,14.0,17.0,14.0,15.0,18.0,29.0,40.0,40,40,
21475,ZWE,Zimbabwe,"Aquatic Products, Other",Food,1000 tonnes,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,


### Del secondo dataset prendiamo tutto e non avremo bisogno di effetuare operazioni di filtraggio

### Estrapolazione degli alimenti il cui inquinamento è tracciato


In [7]:
cibi = Food_Production_df["Food product"]
cibi

0     Wheat & Rye (Bread)
1            Maize (Meal)
2           Barley (Beer)
3                 Oatmeal
4                    Rice
5                Potatoes
6                 Cassava
7              Cane Sugar
8              Beet Sugar
9            Other Pulses
10                   Peas
11                   Nuts
12             Groundnuts
13                Soymilk
14                   Tofu
15            Soybean Oil
16               Palm Oil
17          Sunflower Oil
18           Rapeseed Oil
19              Olive Oil
20               Tomatoes
21         Onions & Leeks
22        Root Vegetables
23              Brassicas
24       Other Vegetables
25           Citrus Fruit
26                Bananas
27                 Apples
28       Berries & Grapes
29                   Wine
30            Other Fruit
31                 Coffee
32         Dark Chocolate
33       Beef (beef herd)
34      Beef (dairy herd)
35          Lamb & Mutton
36               Pig Meat
37           Poultry Meat
38          

### Queste colonne richiedono un aggiusto poichè alcune righe hanno 2 prodotti cosa che non ci consente di effettuare l'incrocio dati quindi procederemo con l'addattare le colonne con il dataset delle Nazioni Unite.

### Inoltre da questi prodotti andremo ad escludere le mandrie di manzi destinati alla produzione di carne (Beef(beef herd)) e le mandrie di manzi destinati alla produzione di latticini (Beef(dairy herd)) che saranno analizzate in separate sede in quanto non sono un prodotto finito vero e proprio ma comunque hanno il loro fattore inquinante.

In [8]:
# Adattamento per favorire l'incrocio dati
cibi_temp = list()
for cibo in cibi:
     if "Beef" not in cibo:
          split = cibo.split('&')
          cibi_temp.append(str.strip(split[0]))
          if len(split) > 1:
               cibi_temp.append(str.strip(split[1]))
cibi_temp.sort()
cibi = pd.Series(cibi_temp)
cibi


0               Apples
1              Bananas
2        Barley (Beer)
3           Beet Sugar
4              Berries
5            Brassicas
6           Cane Sugar
7              Cassava
8               Cheese
9         Citrus Fruit
10              Coffee
11      Dark Chocolate
12                Eggs
13       Fish (farmed)
14              Grapes
15          Groundnuts
16                Lamb
17               Leeks
18        Maize (Meal)
19                Milk
20              Mutton
21                Nuts
22             Oatmeal
23           Olive Oil
24              Onions
25         Other Fruit
26        Other Pulses
27    Other Vegetables
28            Palm Oil
29                Peas
30            Pig Meat
31            Potatoes
32        Poultry Meat
33        Rapeseed Oil
34                Rice
35     Root Vegetables
36         Rye (Bread)
37    Shrimps (farmed)
38         Soybean Oil
39             Soymilk
40       Sunflower Oil
41                Tofu
42            Tomatoes
43         

# Incorcio dei dati
### Ovviamente l'incrocio dei dati porterà ad una diminuzione del campione il che è naturale poichè non vengono tracciati tutti i cibi che ci sono nel datset delle Nazioni Unite

In [9]:
# Funzione per incrocio dati
def incrocio_dati(df, cibi, prodotti_tracciati, colonne) -> None:
     for index, record in FAO_df[FAO_df_colonne_interessate].iterrows():
          food_type = record["Item"].lower()
          # Caso del dizionario
          if(isinstance(cibi, dict)):
               for key in cibi.keys():
                    if food_type.startswith(cibi[key].lower()) and index not in prodotti_tracciati:
                         prodotti_tracciati.append(index)
                         FAO_df.at[index, "Type"] = key
          # Caso lista/pd.series.Series
          elif(isinstance(cibi, pd.core.series.Series) or isinstance(cibi, list)):
               for i in range(len(cibi)):
                    if food_type.startswith(cibi[i].lower()) and index not in prodotti_tracciati:
                         prodotti_tracciati.append(index)
                         FAO_df.at[index, "Type"] = cibi[i]
          # Caso di un tipo non permesso
          else:
               print("No type encoded")


In [10]:
# Inserimento nel df in base al tipo di alimento(pt.1)
prodotti_tracciati = list()
incrocio_dati(FAO_df, cibi, prodotti_tracciati, colonne_interessate)
print("Numero di prodotti tracciati: ", len(prodotti_tracciati))

Numero di prodotti tracciati:  4369


# Dati non presi
### Alcuni dati non sono stati presi a causa della differente nomenclatura tra i datasets in particolare: 'Beet Sugar', 'Cane Sugar', 'Citrus Fruit', 'Fish (farmed)', 'Other Fruit', 'Other Pulses', 'Other Vegetables', 'Pig Meat', 'Sunflower Oil'.
### Effettuerò delle modifiche alle nomenclature al fine di estrapolarli.

In [11]:
colonne_dict = {
     'Beet Sugar': 'sugar beet',
     'Cane Sugar': 'sugar cane',
     'Citrus Fruit': 'citrus',
     'Fish (farmed)': 'fish',
     'Other Fruit': 'fruits, other',
     'Other Pulses': 'pulses, other and product',
     'Other Vegetables': 'vegetables, other',
     'Pig Meat': 'pigmeat',
     'Beef (beef herd)': 'bovine meat'
}

incrocio_dati(FAO_df, colonne_dict, prodotti_tracciati, colonne_interessate)
print("Numero di prodotti tracciati: ", len(prodotti_tracciati))


Numero di prodotti tracciati:  6483


# Creazione del dataset 
### Ho gli indici dei prodotti che vengono tracciati e creo il dataset con i dati giusti

In [12]:
prodotti_tracciati_df = FAO_df.iloc[prodotti_tracciati]
prodotti_tracciati_df = prodotti_tracciati_df[FAO_df_colonne_interessate].reset_index().drop(columns=['index'])
prodotti_tracciati_df[prodotti_tracciati_df["Type"] == "Beef (beef herd)"]


,Area Abbreviation,Area,Item,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,...,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Type
4376,AFG,Afghanistan,Bovine Meat,Food,1000 tonnes,43.0,46.0,47.0,48.0,49.0,...,145.0,121.0,141.0,134.0,136.0,133.0,141.0,153,140,Beef (beef herd)
4385,ALB,Albania,Bovine Meat,Food,1000 tonnes,7.0,7.0,8.0,6.0,7.0,...,48.0,58.0,58.0,64.0,62.0,67.0,67.0,71,71,Beef (beef herd)
4398,DZA,Algeria,Bovine Meat,Food,1000 tonnes,47.0,39.0,29.0,27.0,29.0,...,219.0,195.0,194.0,190.0,208.0,171.0,186.0,222,220,Beef (beef herd)
4408,AGO,Angola,Bovine Meat,Food,1000 tonnes,17.0,19.0,19.0,22.0,25.0,...,154.0,152.0,161.0,156.0,135.0,130.0,143.0,170,181,Beef (beef herd)
4419,ATG,Antigua and Barbuda,Bovine Meat,Food,1000 tonnes,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,Beef (beef herd)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429,VEN,Venezuela (Bolivarian Republic of),Bovine Meat,Food,1000 tonnes,130.0,132.0,140.0,150.0,156.0,...,423.0,527.0,627.0,766.0,727.0,553.0,595.0,646,787,Beef (beef herd)
6440,VNM,Viet Nam,Bovine Meat,Food,1000 tonnes,67.0,68.0,69.0,72.0,75.0,...,257.0,283.0,368.0,441.0,491.0,479.0,645.0,679,682,Beef (beef herd)
6453,YEM,Yemen,Bovine Meat,Food,1000 tonnes,19.0,19.0,19.0,19.0,20.0,...,78.0,78.0,87.0,95.0,102.0,113.0,112.0,114,116,Beef (beef herd)
6465,ZMB,Zambia,Bovine Meat,Food,1000 tonnes,29.0,29.0,29.0,26.0,28.0,...,59.0,58.0,59.0,59.0,68.0,75.0,68.0,69,69,Beef (beef herd)


# Pulizia dei dati
### All'interno dei dati incrociati potremmo avere dei valori nulli che sporcano i dati

## Divisione per alimento

In [13]:
# Dizionario alimento => sub_df
alimenti_dict = dict()
keys = cibi.to_list()

for i in range(len(keys)):
     if keys[i] in colonne_dict.keys():
          keys[i] = colonne_dict[keys[i]].capitalize()

### Una volta fatta la divisione effettuo un filtraggio per la dividere il dataset ed effettuare una pulizia dei prodotto tracciati ed elimino quei dataset che non hanno valori

In [14]:
# Assegnazione delle chiavi a delle liste vuote
for key in keys:
     alimenti_dict[key] = list()

# Divisione dei record in base all'alimento
for index, record in prodotti_tracciati_df.iterrows():
     for key in keys:
          if record["Item"].lower().startswith(key.lower()) and index not in alimenti_dict[key]:
               alimenti_dict[key].append(index)

# Cerco le categorie vuote
prodotti_vuoti = []
for key in alimenti_dict.keys():
     if len(alimenti_dict[key]) == 0:
          prodotti_vuoti.append(key)

# Elimino le categorie di prodotto che hanno una lista di lunghezza 0
for prodotto_vuoto in prodotti_vuoti:
     alimenti_dict.pop(prodotto_vuoto, None)
alimenti_dict.keys()

for alimento in alimenti_dict.keys():
     print(alimento, " -> ", len(alimenti_dict[alimento]))
print("Tipi di alimento trovati: ", list(alimenti_dict.keys()))

Apples  ->  194
Bananas  ->  186
Sugar beet  ->  66
Sugar cane  ->  87
Cassava  ->  225
Citrus  ->  155
Coffee  ->  174
Eggs  ->  360
Fish  ->  762
Grapes  ->  173
Groundnuts  ->  170
Milk  ->  558
Mutton  ->  176
Nuts  ->  173
Olive Oil  ->  174
Onions  ->  167
Fruits, other  ->  185
Pulses, other and product  ->  255
Vegetables, other  ->  258
Palm Oil  ->  124
Peas  ->  196
Pigmeat  ->  169
Potatoes  ->  254
Poultry Meat  ->  175
Rice  ->  273
Tomatoes  ->  175
Wheat  ->  273
Wine  ->  169
Tipi di alimento trovati:  ['Apples', 'Bananas', 'Sugar beet', 'Sugar cane', 'Cassava', 'Citrus', 'Coffee', 'Eggs', 'Fish', 'Grapes', 'Groundnuts', 'Milk', 'Mutton', 'Nuts', 'Olive Oil', 'Onions', 'Fruits, other', 'Pulses, other and product', 'Vegetables, other', 'Palm Oil', 'Peas', 'Pigmeat', 'Potatoes', 'Poultry Meat', 'Rice', 'Tomatoes', 'Wheat', 'Wine']


# Effettuo la pulizia dati
### Elimino le righe cha hanno valori nulli.  

In [15]:
indici_da_eliminare = list()

for key in alimenti_dict.keys():
     for index in alimenti_dict[key]:
          if not prodotti_tracciati_df.iloc[index]["Item"].lower().startswith(key.lower()):
               indici_da_eliminare.append(int(index))

# Pulizia degli indici
for indice_da_eliminare in indici_da_eliminare:
     if indice_da_eliminare in prodotti_tracciati:
          prodotti_tracciati.remove(indice_da_eliminare)

prodotti_tracciati_df = FAO_df.iloc[prodotti_tracciati]
prodotti_tracciati_df = prodotti_tracciati_df[FAO_df_colonne_interessate].reset_index()
prodotti_tracciati_df

prodotti_tracciati_df = prodotti_tracciati_df.dropna().reset_index()
prodotti_tracciati_df.drop(columns=["level_0", "index"], inplace=True)
prodotti_tracciati_df["Prod. 1961 - 2013"] = prodotti_tracciati_df[[f"Y{i}" for i in range(1961, 2014)]].sum(axis=1)
cond = prodotti_tracciati_df["Prod. 1961 - 2013"] != 0
prodotti_tracciati_df = prodotti_tracciati_df[cond]


# Esplorazione dei dati
### Come primo step esplorativo andremo a vedere cosa viene prodotto in quantita' maggiore tra mangimi(Feed) e cibo(Food) in media dal 1961 al 2013.

In [16]:
cibi_mangimi = prodotti_tracciati_df.groupby("Element")[[f"Y{i}" for i in range(1961, 2014)]].mean()
cibi_mangimi

,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
Element,,,,,,,,,,,,,,,,,,,,,
Feed,306.495778,312.610374,322.412545,329.761158,328.675513,342.454765,338.193004,365.063932,362.295537,352.369119,...,457.699638,454.934861,468.605549,467.661037,490.710495,501.890229,505.241255,553.435464,557.232811,568.977081
Food,292.129890,297.916134,305.809512,313.248018,321.925339,329.274866,334.969573,347.122475,353.654053,361.072360,...,865.456405,886.113782,907.593710,935.982102,957.513168,975.363334,1000.261059,1025.631041,1050.720020,1064.981335


## Anno di maggiore produzione di cibi e mangimi
### Tutti i valori vanno moltiplicati per 1000

In [17]:
magimi_max_produzione = cibi_mangimi.loc["Feed"].max()
cibo_max_produzione = cibi_mangimi.loc["Food"].max()
print(cibi_mangimi.loc["Feed"].max() * 1000, " tonnellate di mangime nell'anno", cibi_mangimi.idxmax(axis=1)[0].split("Y")[1])
print(cibi_mangimi.loc["Food"].max() * 1000, "tonnellate di cibo nell'anno", cibi_mangimi.idxmax(axis=1)[1].split("Y")[1])

568977.0808202655  tonnellate di mangime nell'anno 2013
1064981.3346970084 tonnellate di cibo nell'anno 2013


### I valori più elevati corrispondono con il 2013 che è anche l'ultimo anno a disposizione.
# Incremento produzione cibi e mangimi
### Vedremo l'aumento della produzione di cibi anno per anno, ogni colonna ci dice di quanto è aumentata la produzione rispetto all'anno precedente.
# Cibi

In [18]:
Food = cibi_mangimi.loc["Food"]

Food_yearly_increment = [abs(Food[f"Y{i}"] - Food[f"Y{i+1}"]) for i in range(1961, 2014) if i + 1 < 2014]
Food_yearly_increment_df = pd.DataFrame()
for i in range(1961, 2014):
     if i+1 < 2014:
          Food_yearly_increment_df[f"Y{i}/Y{i+1}"] = pd.Series(Food_yearly_increment[abs(1961 - i)])

print(f"Incremento annuo medio della produzione di cibo = {Food_yearly_increment_df.iloc[0].mean() * 1000} tonnellate")
Food_yearly_increment_df * 1000

Incremento annuo medio della produzione di cibo = 14862.527781602186 tonnellate


,Y1961/Y1962,Y1962/Y1963,Y1963/Y1964,Y1964/Y1965,Y1965/Y1966,Y1966/Y1967,Y1967/Y1968,Y1968/Y1969,Y1969/Y1970,Y1970/Y1971,...,Y2003/Y2004,Y2004/Y2005,Y2005/Y2006,Y2006/Y2007,Y2007/Y2008,Y2008/Y2009,Y2009/Y2010,Y2010/Y2011,Y2011/Y2012,Y2012/Y2013
0,5786.243927,7893.377653,7438.506776,8677.320378,7349.526975,5694.707236,12152.902071,6531.577602,7418.307338,7535.157249,...,19188.954232,20657.37663,21479.928407,28388.391716,21531.066223,17850.166198,24897.724367,25369.982102,25088.979801,14261.314242


# Mangimi

In [19]:
Feed = cibi_mangimi.loc["Feed"]

Feed_yearly_increment = [abs(Feed[f"Y{i}"] - Feed[f"Y{i+1}"]) for i in range(1961, 2014) if i + 1 < 2014]
Feed_yearly_increment_df = pd.DataFrame()
for i in range(1961, 2014):
     if i+1 < 2014:
          Feed_yearly_increment_df[f"Y{i}/Y{i+1}"] = pd.Series(Feed_yearly_increment[abs(1961 - i)])

print(f"Incremento annuo medio della produzione di mangimi = {Feed_yearly_increment_df.iloc[0].mean() * 1000} tonnellate")
Feed_yearly_increment_df * 1000

Incremento annuo medio della produzione di mangimi = 11787.02329034054 tonnellate


,Y1961/Y1962,Y1962/Y1963,Y1963/Y1964,Y1964/Y1965,Y1965/Y1966,Y1966/Y1967,Y1967/Y1968,Y1968/Y1969,Y1969/Y1970,Y1970/Y1971,...,Y2003/Y2004,Y2004/Y2005,Y2005/Y2006,Y2006/Y2007,Y2007/Y2008,Y2008/Y2009,Y2009/Y2010,Y2010/Y2011,Y2011/Y2012,Y2012/Y2013
0,6114.595899,9802.171291,7348.612786,1085.645356,13779.252111,4261.761158,26870.92883,2768.395657,9926.41737,13924.004825,...,23575.392039,2764.77684,13670.687575,944.51146,23049.457177,11179.73462,3351.025332,48194.209891,3797.3462,11744.270205


# Cibi piu' e meno prodotti dal 1961 al 2013
### Scopriremo quali sono i 5 cibi piu' prodotti ed i 5 cibi meno prodotti dal 1961 al 2013.

In [20]:
columns_interested = ["Item", "Element", "Area", "Prod. 1961 - 2013", "Type"]
Food_items = prodotti_tracciati_df[columns_interested]
Food_items = Food_items[Food_items["Element"] == "Food"]
Food_items = Food_items.sort_values(by=["Prod. 1961 - 2013"], ascending=False).reset_index().drop(columns=["index"]).drop_duplicates().reset_index().drop(columns=["index"])
Food_items["Tonnes"] = Food_items["Prod. 1961 - 2013"] * 1000
Food_items.groupby("Type").sum(numeric_only=True).sort_values(by="Tonnes", ascending=False)

,Prod. 1961 - 2013,Tonnes
Type,,
Milk,17904347.0,1.790435e+10
Other Vegetables,17755357.0,1.775536e+10
Wheat,14780040.0,1.478004e+10
Rice,13498424.0,1.349842e+10
Potatoes,6353554.0,6.353554e+09
Other Fruit,4348591.0,4.348591e+09
Cassava,3465370.0,3.465370e+09
Fish (farmed),3463336.0,3.463336e+09
Tomatoes,3219400.0,3.219400e+09


# Mangimi più e meno prodotti dal 1961 al 2013
### Scopriremo quali sono i 5 mangimi più prodotti ed i 5 mangimi meno prodotti dal 1961 al 2013.

In [21]:
Feed_items = prodotti_tracciati_df[columns_interested]
Feed_items = Feed_items[Feed_items["Element"] == "Feed"]
Feed_items = Feed_items.sort_values(by=["Prod. 1961 - 2013"], ascending=False).reset_index().drop(columns=["index"]).drop_duplicates().reset_index().drop(columns=["index"])
Feed_items["Tonnes"] = Feed_items["Prod. 1961 - 2013"] * 1000
Feed_items.groupby("Type").sum(numeric_only=True).sort_values(by="Tonnes", ascending=False)

,Prod. 1961 - 2013,Tonnes
Type,,
Milk,2839405.0,2.839405e+09
Wheat,2837106.0,2.837106e+09
Cassava,2350240.0,2.350240e+09
Potatoes,2021093.0,2.021093e+09
Cane Sugar,1146278.0,1.146278e+09
Fish (farmed),1075008.0,1.075008e+09
Rice,962161.0,9.621610e+08
Other Vegetables,672160.0,6.721600e+08
Other Pulses,238262.0,2.382620e+08


# Si va a calcolare l'inquinamento dei cibi più prodotti a livello mondiale

#### Passaggio da tonnellate a KG

In [22]:
# Prendo le colonne relative all'inquinamento da CO2
cibo_max_produzione = Food_items.iloc[:5]
cibo_max_type = Food_items.iloc[:5]["Type"].to_list()
cibo_max_type
# Passaggio da tonnellate in KG bisogna moltimplicare per 1000
Food_CO2_total_emissions = [{record['Food product']: record['Total_emissions']} for index, record in Food_Production_df.iterrows() if record["Food product"] in cibo_max_type  or record["Food product"] == "Wheat & Rye (Bread)"]
KGCO2 = cibo_max_produzione["Tonnes"] * 1000
cibo_max_produzione = cibo_max_produzione.assign(KGCO2=KGCO2)
cibo_max_produzione

,Item,Element,Area,Prod. 1961 - 2013,Type,Tonnes,KGCO2
0,"Vegetables, Other",Food,"China, mainland",7962498.0,Other Vegetables,7.962498e+09,7.962498e+12
1,Rice (Milled Equivalent),Food,"China, mainland",4271067.0,Rice,4.271067e+09,4.271067e+12
2,Wheat and products,Food,"China, mainland",3521951.0,Wheat,3.521951e+09,3.521951e+12
3,Milk - Excluding Butter,Food,United States of America,3353075.0,Milk,3.353075e+09,3.353075e+12
4,Rice (Milled Equivalent),Food,India,3096326.0,Rice,3.096326e+09,3.096326e+12


#### Aggiungo l'inquinamento da CO2 in KG e tonnellate

In [23]:

for index, record in cibo_max_produzione.iterrows():
     for CO2_record in Food_CO2_total_emissions:
          key = [key for key in CO2_record.keys()][0]
          if record["Type"] in key:
               cibo_max_produzione.at[index, "KGCO2"] *= CO2_record[key]
               cibo_max_produzione.at[index, "TonnesCO2"] = cibo_max_produzione.at[index, "Tonnes"] * CO2_record[key]
cibo_max_produzione

,Item,Element,Area,Prod. 1961 - 2013,Type,Tonnes,KGCO2,TonnesCO2
0,"Vegetables, Other",Food,"China, mainland",7962498.0,Other Vegetables,7.962498e+09,3.981249e+12,3.981249e+09
1,Rice (Milled Equivalent),Food,"China, mainland",4271067.0,Rice,4.271067e+09,1.708427e+13,1.708427e+10
2,Wheat and products,Food,"China, mainland",3521951.0,Wheat,3.521951e+09,4.930731e+12,4.930731e+09
3,Milk - Excluding Butter,Food,United States of America,3353075.0,Milk,3.353075e+09,9.388610e+12,9.388610e+09
4,Rice (Milled Equivalent),Food,India,3096326.0,Rice,3.096326e+09,1.238530e+13,1.238530e+10


#### I cibi sopra elencati con il rispettivo inquinamento da CO2 sono cibi classificabili come vegetariani. Ora adremo ad effettuare la stessa operazione ma per le carni.

In [24]:
meat_conditions = (Food_items["Item"] == "Bovine Meat") | (Food_items["Item"] == "Pigmeat") | (Food_items["Item"] == "Poultry Meat")
meats_df = Food_items[meat_conditions]
meats_type = meats_df["Type"].drop_duplicates().to_list()
worlds_meat_production_df = meats_df.groupby("Type").sum(numeric_only=True)
meats_KGCO2 = [{index: Food_Production_df[Food_Production_df["Food product"] == index]["Total_emissions"].to_list()[0]} for index, record in worlds_meat_production_df.iterrows()]
worlds_meat_production_df
meats_KGCO2
for record in meats_KGCO2:
    for key, value in record.items():
        worlds_meat_production_df.at[key, "KGCO2"] =  worlds_meat_production_df.at[key, "Tonnes"] * record[key] * 1000
        worlds_meat_production_df.at[key, "TonnesCO2"] = worlds_meat_production_df.at[key, "Tonnes"] * record[key]

worlds_meat_production_df


,Prod. 1961 - 2013,Tonnes,KGCO2,TonnesCO2
Type,,,,
Beef (beef herd),2268633.0,2.268633e+09,1.352105e+14,1.352105e+11
Pig Meat,3037100.0,3.037100e+09,2.186712e+13,2.186712e+10
Poultry Meat,2118056.0,2.118056e+09,1.292014e+13,1.292014e+10


# Calcolo l'inquinamento di tutti i cibi e mangimi sia in tonnellate che in KG

In [25]:
# Estrazione dei tipi di alimento
Food_types = Food_items["Type"].drop_duplicates().to_list()
Food_types_CO2KG = [Food_Production_df[Food_Production_df["Food product"] == food_type]["Total_emissions"].to_list() for food_type in Food_types]
# Creazione del dizionario
Food_types_CO2KG_dict = dict()
for i in range(len(Food_types)):
     Food_types_CO2KG_dict[Food_types[i]] = 0

     if len(Food_types_CO2KG[i]) != 0:
          Food_types_CO2KG_dict[Food_types[i]] = Food_types_CO2KG[i][0]

for index, record in Food_items.iterrows():
     Food_type = record["Type"]
     if Food_type in Food_types_CO2KG_dict.keys():
          Food_items.at[index, "KGCO2"] =  Food_items.at[index, "Tonnes"] * Food_types_CO2KG_dict[Food_type] * 1000
          Food_items.at[index, "TonnesCO2"] = Food_items.at[index, "Tonnes"] * Food_types_CO2KG_dict[Food_type]

Food_items.groupby("Type").sum(numeric_only=True).sort_values(by="KGCO2", ascending=False)

,Prod. 1961 - 2013,Tonnes,KGCO2,TonnesCO2
Type,,,,
Beef (beef herd),2268633.0,2.268633e+09,1.352105e+14,1.352105e+11
Rice,13498424.0,1.349842e+10,5.399370e+13,5.399370e+10
Milk,17904347.0,1.790435e+10,5.013217e+13,5.013217e+10
Pig Meat,3037100.0,3.037100e+09,2.186712e+13,2.186712e+10
Fish (farmed),3463336.0,3.463336e+09,1.766301e+13,1.766301e+10
Poultry Meat,2118056.0,2.118056e+09,1.292014e+13,1.292014e+10
Other Vegetables,17755357.0,1.775536e+10,8.877678e+12,8.877678e+09
Eggs,1622053.0,1.622053e+09,7.299238e+12,7.299238e+09
Tomatoes,3219400.0,3.219400e+09,4.507160e+12,4.507160e+09


# Calcolo eutrofizzazione dei cibi e mangimi prodotti a livello mondiale prodotti

#### L'eutrofizzazione è un fenomeno dato dal rilascio della sostanza PO4 creerò una colonna che calcola i grammi di PO4 per ogni kilo di cibo/mangime prodotto

In [26]:
Food_types_PO4_emissions = [Food_Production_df[Food_Production_df["Food product"] == food_type]["Eutrophying emissions per kilogram (gPO₄eq per kilogram)"].to_list() for food_type in Food_types]
Food_types_PO4_dict = dict()
for i in range(len(Food_types)):
     Food_types_PO4_dict[Food_types[i]] = 0

     if len(Food_types_PO4_emissions[i]) != 0:
          Food_types_PO4_dict[Food_types[i]] = Food_types_PO4_emissions[i][0]

for index, record in Food_items.iterrows():
     Food_type = record["Type"]
     if Food_type in Food_types_PO4_dict.keys():
          Food_items.at[index, "Total gPO4 per kilogram"] =  Food_items.at[index, "Tonnes"] * 1000000 * Food_types_PO4_dict[Food_type]

Food_items.groupby("Type").sum(numeric_only=True).sort_values(by="Total gPO4 per kilogram", ascending=False)


,Prod. 1961 - 2013,Tonnes,KGCO2,TonnesCO2,Total gPO4 per kilogram
Type,,,,,
Fish (farmed),3463336.0,3.463336e+09,1.766301e+13,1.766301e+10,8.142996e+17
Beef (beef herd),2268633.0,2.268633e+09,1.352105e+14,1.352105e+11,6.837887e+17
Rice,13498424.0,1.349842e+10,5.399370e+13,5.399370e+10,4.733897e+17
Pig Meat,3037100.0,3.037100e+09,2.186712e+13,2.186712e+10,2.319737e+17
Milk,17904347.0,1.790435e+10,5.013217e+13,5.013217e+10,1.906813e+17
Poultry Meat,2118056.0,2.118056e+09,1.292014e+13,1.292014e+10,1.031493e+17
Other Vegetables,17755357.0,1.775536e+10,8.877678e+12,8.877678e+09,4.030466e+16
Eggs,1622053.0,1.622053e+09,7.299238e+12,7.299238e+09,3.529587e+16
Coffee,272799.0,2.727990e+08,4.501184e+12,4.501184e+09,3.014975e+16


# Calcolo acqua usata per i cibi/mangimi prodotti a livello mondiale

In [27]:
Food_types_water_liters_per_kilogram = [Food_Production_df[Food_Production_df["Food product"] == food_type]["Freshwater withdrawals per kilogram (liters per kilogram)"].to_list() for food_type in Food_types]
Food_types_water_liters_per_kilogram_dict = dict()
for i in range(len(Food_types)):
     Food_types_water_liters_per_kilogram_dict[Food_types[i]] = 0

     if len(Food_types_water_liters_per_kilogram[i]) != 0:
          Food_types_water_liters_per_kilogram_dict[Food_types[i]] = Food_types_water_liters_per_kilogram[i][0]

for index, record in Food_items.iterrows():
     Food_type = record["Type"]
     if Food_type in Food_types_water_liters_per_kilogram_dict.keys():
          Food_items.at[index, "Total water(lt) per kilogram"] =  Food_items.at[index, "Tonnes"] * 1000 * Food_types_water_liters_per_kilogram_dict[Food_type]

Food_items.groupby("Type").sum(numeric_only=True).sort_values(by="Total water(lt) per kilogram", ascending=False)

,Prod. 1961 - 2013,Tonnes,KGCO2,TonnesCO2,Total gPO4 per kilogram,Total water(lt) per kilogram
Type,,,,,,
Rice,13498424.0,1.349842e+10,5.399370e+13,5.399370e+10,4.733897e+17,3.034986e+16
Fish (farmed),3463336.0,3.463336e+09,1.766301e+13,1.766301e+10,8.142996e+17,1.278421e+16
Milk,17904347.0,1.790435e+10,5.013217e+13,5.013217e+10,1.906813e+17,1.124751e+16
Pig Meat,3037100.0,3.037100e+09,2.186712e+13,2.186712e+10,2.319737e+17,5.454024e+15
Beef (beef herd),2268633.0,2.268633e+09,1.352105e+14,1.352105e+11,6.837887e+17,3.292240e+15
Other Vegetables,17755357.0,1.775536e+10,8.877678e+12,8.877678e+09,4.030466e+16,1.819924e+15
Poultry Meat,2118056.0,2.118056e+09,1.292014e+13,1.292014e+10,1.031493e+17,1.397917e+15
Nuts,329054.0,3.290540e+08,6.581080e+10,6.581080e+07,6.301384e+15,1.360243e+15
Tomatoes,3219400.0,3.219400e+09,4.507160e+12,4.507160e+09,2.417769e+16,1.190534e+15


# Calcolo emissione gas serra prodotti dal 1961 al 2013

#### Per questo calcolo andrò a inserire per ogni prodotto tracciato il numero di kcal presenti in 100gr di cibo/mangime uso google come fonte per i miei dati.
#### Verrà create una nuova colonna chiamata "Kg producted x 1000kcal" che ci dirà quanti chili devono essere prodotti per generare 1000kcal

In [33]:
Food_kcalories_100gr = {

     "Other Vegetables": 65,
     "Rice": 130,
     "Wheat": 329,
     "Milk": 42,
     "Potatoes": 77,
     "Pig Meat": 242,
     "Fish (farmed)": 158,
     "Other Fruit": 80,
     "Tomatoes": 18,
     "Beef (beef herd)": 332,
     "Cassava": 159,
     "Eggs": 155,
     "Poultry Meat": 272,
     "Cane Sugar": 400,
     "Bananas": 89,
     "Apples": 52,
     "Other Pulses": 181,
     "Onions": 40,
     "Wine": 83,
     "Grapes": 67,
     "Citrus Fruit": 72,
     "Groundnuts": 567,
     "Mutton": 294,
     "Coffee": 0,
     "Peas": 81,
     "Palm Oil": 884,
     "Nuts": 607,
     "Olive Oil": 884,
     "Beet Sugar": 405,

}

# Aggiungo la colonna a Food_Production_df
for index, record in Food_Production_df.iterrows():
     Type = record["Food product"]
     if Type in Food_kcalories_100gr.keys():
          if Food_kcalories_100gr[Type] != 0 and Food_kcalories_100gr[Type] != np.NAN:
               Food_Production_df.at[index, "Kg producted x 1000kcal"] = (1000 / (Food_kcalories_100gr[Type]))

gas_serra_emissions = Food_items["Tonnes"] * 1000**2 # Con questa operazione porto tutto alla grandezza dei grammi
for index, record in Food_items.iterrows():
     Type = record["Type"]
     if Type in Food_kcalories_100gr.keys():
          if len(Food_Production_df[Food_Production_df["Food product"] == Type]["Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)"].values) != 0:
               Food_items.at[index, "KGCO2eq tot"] = ((record["Tonnes"] * 1000)/(Food_Production_df[Food_Production_df["Food product"] == "Rice"]["Kg producted x 1000kcal"]))
          else:
               Food_items.at[index, "KGCO2eq tot"] = 0
               # Food_items.at[index, "KGCO2eq tot"] *= Food_Production_df[Food_Production_df["Food product"] == Type]["Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)"]
# print(Food_Production_df[Food_Production_df["Food product"] == "Rice"]["Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)"].values[0])
# str(Food_Production_df[Food_Production_df["Food product"] == "Other Vegetables"]["Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal)"].values[0]) == "nan"
Food_items.groupby("Item").sum(numeric_only=True).sort_values(by="KGCO2eq tot", ascending=False)

,Prod. 1961 - 2013,Tonnes,KGCO2,TonnesCO2,Total gPO4 per kilogram,Total water(lt) per kilogram,KGCO2eq tot
Item,,,,,,,
Milk - Excluding Butter,17904347.0,1.790435e+10,5.013217e+13,5.013217e+10,1.906813e+17,1.124751e+16,2.327565e+12
"Vegetables, Other",17755357.0,1.775536e+10,8.877678e+12,8.877678e+09,4.030466e+16,1.819924e+15,2.308196e+12
Rice (Milled Equivalent),13476506.0,1.347651e+10,5.390602e+13,5.390602e+10,4.726211e+17,3.030058e+16,1.751946e+12
Potatoes and products,6353554.0,6.353554e+09,1.906066e+12,1.906066e+09,2.211037e+16,3.754950e+14,8.259620e+11
"Fruits, Other",4348591.0,4.348591e+09,3.044014e+12,3.044014e+09,1.056708e+16,6.675087e+14,5.653168e+11
Cassava and products,3465370.0,3.465370e+09,3.118833e+12,3.118833e+09,2.391105e+15,0.000000e+00,4.504981e+11
"Fish, Seafood",3455379.0,3.455379e+09,1.762243e+13,1.762243e+10,8.124287e+17,1.275484e+16,4.491993e+11
Tomatoes and products,3219400.0,3.219400e+09,4.507160e+12,4.507160e+09,2.417769e+16,1.190534e+15,4.185220e+11
Pigmeat,3037100.0,3.037100e+09,2.186712e+13,2.186712e+10,2.319737e+17,5.454024e+15,3.948230e+11
